In [1]:
from src.data.tushare.download.daily import DailyLimit
import time
dl = DailyLimit()
while len(dl.update_dates()) > 3:
    try:
        dl.update()
    except Exception as e:
        print(e)
        wait_seconds = 10
        print(f'wait {wait_seconds} seconds')
        time.sleep(wait_seconds)


DailyLimit Updating trade_ts/day_limit at 20070102
DailyLimit Updating trade_ts/day_limit at 20070103
DailyLimit Updating trade_ts/day_limit at 20070104
DailyLimit Updating trade_ts/day_limit at 20070105
DailyLimit Updating trade_ts/day_limit at 20070106
DailyLimit Updating trade_ts/day_limit at 20070107
DailyLimit Updating trade_ts/day_limit at 20070108
DailyLimit Updating trade_ts/day_limit at 20070109
DailyLimit Updating trade_ts/day_limit at 20070110
DailyLimit Updating trade_ts/day_limit at 20070111
DailyLimit Updating trade_ts/day_limit at 20070112
DailyLimit Updating trade_ts/day_limit at 20070113
DailyLimit Updating trade_ts/day_limit at 20070114
DailyLimit Updating trade_ts/day_limit at 20070115
DailyLimit Updating trade_ts/day_limit at 20070116
DailyLimit Updating trade_ts/day_limit at 20070117
DailyLimit Updating trade_ts/day_limit at 20070118
DailyLimit Updating trade_ts/day_limit at 20070119
DailyLimit Updating trade_ts/day_limit at 20070120
DailyLimit Updating trade_ts/da

In [1]:
from src.data.tushare.model.cne5 import TuShareCNE5_Calculator

task_cne5 = TuShareCNE5_Calculator()
task_cne5.calc_specific_risk(20241104)

,secid,spec_risk
0,1,0.012616
1,2,0.016491
2,4,0.076816
3,6,0.058301
4,7,0.058771
...,...,...
5298,873339,0.084023
5299,873527,0.075710
5300,873576,NaN
5301,873593,NaN


In [3]:
import numpy as np
import pandas as pd

from typing import Literal
from src.factor.classes import StockFactorCalculator
from src.data import TSData
from src.func.transform import apply_ols
from src.func.singleton import singleton

date = 20241104
val = TSData.TRADE.get_val(date).set_index('secid')
np.log(val['free_share'] * val['close'])

secid
1         25.261434
2         24.796052
4         21.277646
6         22.713214
7         21.138546
            ...    
920019    20.730142
920066    19.963528
920088    20.085129
920099    20.126759
920118    19.544047
Length: 5349, dtype: float64

In [1]:
from src_factor_definition.level0.liquidity_cap import lncap , lncap_liq , lncap_free

date = 20241104
df = lncap()(date)
obj1 , obj2 = lncap() , lncap_liq()

In [2]:
obj1 , obj2

(lncap(from 20070101 , behavior , liquidity),
 lncap_liq(from 20070101 , behavior , liquidity))

In [11]:
import pandas as pd
from src.basic import PATH
df = pd.read_feather(PATH.get_target_path('trade_ts' , 'min' , date = 20241104 , makedir = True))
df


,secid,minute,open,high,low,close,amount,volume,vwap,num_trades
0,1,0,11.43,11.46,11.39,11.40,48944566.0,4283866.0,11.425326,2026.0
1,1,1,11.40,11.40,11.33,11.35,44207522.0,3893700.0,11.353602,1463.0
2,1,2,11.36,11.36,11.31,11.33,44734953.0,3947900.0,11.331329,1604.0
3,1,3,11.34,11.34,11.31,11.31,24852773.0,2195600.0,11.319354,879.0
4,1,4,11.31,11.32,11.26,11.29,45381928.0,4019319.0,11.290950,1876.0
...,...,...,...,...,...,...,...,...,...,...
1225675,689009,235,45.36,45.37,45.33,45.37,3226637.0,71143.0,45.354244,94.0
1225676,689009,236,45.40,45.42,45.38,45.41,4037872.0,88934.0,45.403018,81.0
1225677,689009,237,45.39,45.39,45.39,45.39,69413.0,1529.0,45.397646,6.0
1225678,689009,238,45.39,45.39,45.39,45.39,0.0,0.0,45.390000,0.0


In [6]:
obj3 = obj2.calculate(date + 1)
obj3


lncap_liq(from 20070101 , behavior , liquidity)

In [9]:
obj3.factors

{20241104:         lncap_liq
 secid            
 1       26.127691
 2       25.202108
 4       21.579386
 6       23.175123
 7       21.511591
 ...           ...
 920019  20.730142
 920066  19.963528
 920088  20.085129
 920099  20.126759
 920118  19.707483
 
 [5349 rows x 1 columns],
 20241105:         lncap_liq
 secid            
 1       26.144135
 2       25.231442
 4       21.622437
 6       23.185534
 7       21.606646
 ...           ...
 920019  20.792325
 920066  20.074198
 920088  20.150895
 920099  20.191704
 920118  19.771307
 
 [5352 rows x 1 columns]}

In [4]:
obj3 is obj2

True

In [5]:
obj1 is obj2

False

In [4]:
import numpy as np
import pandas as pd

from typing import Literal
from src.factor.classes import StockFactorCalculator
from src.data import TSData
from src.func.transform import apply_ols
from src.func.singleton import singleton

def cap_classic(date , cap_type : Literal['tt' , 'fl' , 'fr']):
    val = TSData.TRADE.get_val(date).set_index('secid')
    if cap_type == 'tt':
        return np.log(val['total_share'] * val['close'])
    elif cap_type == 'fl':
        return np.log(val['float_share'] * val['close'])
    elif cap_type == 'fr':
        return np.log(val['free_share'] * val['close'])
    else:
        raise ValueError(f'cap_type {cap_type} not supported')

@singleton
class lncap(StockFactorCalculator):
    init_date = 20070101
    category0 = 'behavior'
    category1 = 'liquidity'
    description = '市值(对数总市值)'
    
    def calc_factor(self, date: int):
        return cap_classic(date , 'tt')
    
@singleton
class lncap_liq(StockFactorCalculator):
    init_date = 20070101
    category0 = 'behavior'
    category1 = 'liquidity'
    description = '市值(对数流通市值)'
    
    def calc_factor(self, date: int):
        return cap_classic(date , 'fl')
    
@singleton
class lncap_free(StockFactorCalculator):
    init_date = 20070101
    category0 = 'behavior'
    category1 = 'liquidity'
    description = '市值(对数自由流通市值)'
    
    def calc_factor(self, date: int):
        return cap_classic(date , 'fr')
    
lncap_free().calc_factor(20241104)

secid
1         25.261434
2         24.796052
4         21.277646
6         22.713214
7         21.138546
            ...    
920019    20.730142
920066    19.963528
920088    20.085129
920099    20.126759
920118    19.544047
Length: 5349, dtype: float64

In [13]:
import tushare as ts

pro = ts.pro_api('08232196bbac2a4ec51587784169f501f6387de3ad2fbe7719b5444a')
df = ts.pro_bar(api = pro , freq='1min', 
                start_date='2024-11-05 00:00:00', 
                end_date='2024-11-05 17:00:00')

df

抱歉，您每天最多访问该接口2次，权限的具体详情访问：https://tushare.pro/document/1?doc_id=108。
抱歉，您每天最多访问该接口2次，权限的具体详情访问：https://tushare.pro/document/1?doc_id=108。
抱歉，您每分钟最多访问该接口2次，权限的具体详情访问：https://tushare.pro/document/1?doc_id=108。


OSError: ERROR.

In [14]:
df

,ts_code,trade_time,close,open,high,low,vol,amount,trade_date,pre_close,change,pct_chg
0,600000.SH,2024-11-05 15:00:00,10.06,10.06,10.06,10.06,1196900.0,12040814.0,20241105,10.06,0.00,0.0
1,600000.SH,2024-11-05 14:59:00,10.06,10.06,10.06,10.06,0.0,0.0,20241105,10.06,0.00,0.0
2,600000.SH,2024-11-05 14:58:00,10.06,10.06,10.06,10.06,2000.0,20120.0,20241105,10.05,0.01,0.1
3,600000.SH,2024-11-05 14:57:00,10.05,10.05,10.06,10.04,185072.0,1860740.0,20241105,10.05,0.00,0.0
4,600000.SH,2024-11-05 14:56:00,10.05,10.05,10.06,10.04,200823.0,2018952.0,20241105,10.06,-0.01,-0.1
...,...,...,...,...,...,...,...,...,...,...,...,...
236,600000.SH,2024-11-05 09:34:00,9.97,9.98,9.98,9.95,193258.0,1925624.0,20241105,9.97,0.00,0.0
237,600000.SH,2024-11-05 09:33:00,9.97,9.99,9.99,9.97,204800.0,2044846.0,20241105,9.99,-0.02,-0.2
238,600000.SH,2024-11-05 09:32:00,9.99,10.00,10.01,9.99,374486.0,3744104.0,20241105,9.99,0.00,0.0
239,600000.SH,2024-11-05 09:31:00,9.99,9.95,10.00,9.93,473144.0,4715288.0,20241105,9.97,0.02,0.2


In [1]:
from src.data.tushare.model.cne5 import TuShareCNE5_Calculator
cne5 = TuShareCNE5_Calculator()
cne5.Update('exposure')
cne5.Update('risk')


In [2]:
from src.data.tushare.download.daily import DailyMoneyFlow
DailyMoneyFlow().get_data(20241104)

,secid,buy_sm_vol,buy_sm_amount,sell_sm_vol,sell_sm_amount,buy_md_vol,buy_md_amount,sell_md_vol,sell_md_amount,buy_lg_vol,buy_lg_amount,sell_lg_vol,sell_lg_amount,buy_elg_vol,buy_elg_amount,sell_elg_vol,sell_elg_amount,net_mf_vol,net_mf_amount
0,1,206641,23472.72,214673,24397.62,294219,33423.61,300228,34133.57,349891,39772.04,351405,39933.79,280765,31919.03,265211,30122.43,-66602,-7439.96
1,2,557125,49903.84,383794,34410.17,636053,56992.08,573613,51429.29,496666,44509.93,574776,51486.75,247204,22159.22,404864,36238.86,-182446,-16153.23
2,4,75115,13725.40,64391,11822.04,45394,8307.64,52668,9630.07,26685,4909.84,24742,4527.00,3369,622.08,8762,1585.84,2299,428.43
3,6,286767,24512.43,197046,16857.81,292197,24981.22,317110,27118.47,230250,19685.66,277444,23679.77,96853,8279.79,114466,9803.05,-93641,-7936.06
4,7,47080,3313.73,54514,3838.73,52907,3730.79,44947,3173.40,29532,2082.85,30665,2159.18,2176,154.71,1569,110.77,27082,1912.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5088,688799,3813,1498.85,3640,1431.49,5319,2090.90,5359,2107.46,1958,770.76,2090,821.56,0,0.00,0,0.00,-763,-297.29
5089,688800,24307,8921.19,25042,9211.46,18769,6904.80,19942,7339.27,11353,4183.04,7961,2920.42,2903,1077.74,4387,1615.61,2069,773.56
5090,688819,29010,8197.77,28051,7928.01,20181,5708.49,19625,5551.70,8531,2411.08,10370,2932.08,992,282.18,668,187.73,-4120,-1163.00
5091,688981,72765,62502.24,84901,72982.84,181190,155670.85,183621,157781.59,215116,184889.83,220352,189364.82,107512,92388.61,87708,75322.27,12858,11539.99


In [50]:
import numpy as np
import pandas as pd

from dataclasses import dataclass

from src.factor.classes import StockFactorCalculator
from src.data import TSData
from src.func.transform import time_weight , apply_ols
from src.func.singleton import singleton

@dataclass
class FF3_Model:
    stk : pd.DataFrame
    mkt : pd.DataFrame | pd.Series
    hml : pd.DataFrame | pd.Series
    smb : pd.DataFrame | pd.Series

    def __post_init__(self):
        ...

    def fit(self):
        ff3_x = self.mkt.merge(self.hml , on = 'date').merge(self.smb , on = 'date')
        ff3_x.columns = ['mkt' , 'hml' , 'smb']
        ff3_x['intercept'] = 1.
        self.ff3_x = ff3_x.loc[:,['intercept' , 'mkt' , 'hml' , 'smb']]

        self.coef = apply_ols(self.ff3_x , self.stk , intercept=False)
        self.pred = self.ff3_x.values @ self.coef
        self.resid = self.stk - self.pred
        self.r2    = 1 - (self.resid ** 2).sum() / (self.stk ** 2).sum()
        return self
    
class FamaFrench3:
    N_MONTHS : int = -1
    def __init__(self , date):
        assert self.N_MONTHS > 0 , self.N_MONTHS
        if getattr(self, 'date' , None) is None or self.date != int(date):
            self.date = int(date)
            self.fit(self.date)

    @staticmethod
    def group_ret(df : pd.DataFrame):
        df = df.groupby('date')[['mv_add' , 'mv']].sum()
        return df['mv_add'] / df['mv']

    @classmethod
    def mkt(cls , df : pd.DataFrame):
        return cls.group_ret(df).rename('mkt')

    @classmethod
    def smb(cls , df : pd.DataFrame):
        mv_rank = df['mv'].groupby('date').rank(pct = True)
        ret_B = cls.group_ret(df[mv_rank >= 0.9])
        ret_S = cls.group_ret(df[mv_rank < 0.5])
        return (ret_S - ret_B).rename('smb')
    
    @classmethod
    def hml(cls , df : pd.DataFrame):
        bp_rank = df['bp'].groupby('date').rank(pct = True)
        ret_H = cls.group_ret(df[bp_rank >= 2/3])
        ret_L = cls.group_ret(df[bp_rank < 1/3])
        return (ret_H - ret_L).rename('hml')

    def fit(self , date , half_life = 0 , min_finite_ratio = 0.25):
        start_date , end_date = TSData.CALENDAR.td_start_end(date , self.N_MONTHS , 'm')

        rets = TSData.TRADE.get_returns(start_date , end_date , mask = False , pivot = False).rename(columns={'pctchange':'ret'})
        mv   = TSData.TRADE.get_mv(start_date , end_date , mv_type = 'circ_mv' , pivot = False , prev=True).rename(columns={'circ_mv':'mv'})
        btop = (1 / TSData.TRADE.get_val_data(start_date , end_date , cols = 'pb' , pivot = False , prev = True)).rename(columns={'pb':'bp'})
        rets = rets.merge(mv , on = ['date' , 'secid']).merge(btop , on = ['date' , 'secid'])
        rets['mv_add']= rets['mv'] * rets['ret']
        stk = rets.pivot_table('ret' , 'date' , 'secid')
        mkt = self.mkt(rets).to_frame()
        hml = self.hml(rets)
        smb = self.smb(rets)

        self.model = FF3_Model(stk , mkt , hml , smb).fit()
        return self
    
    @staticmethod
    def select_ff3(n_months : int):
        if n_months == 1:
            return FF3_1m
        elif n_months == 2:
            return FF3_2m
        elif n_months == 3:
            return FF3_3m
        elif n_months == 6:
            return FF3_6m
        elif n_months == 12:
            return FF3_12m
        else:
            raise ValueError(f'n_months must be in [1, 2, 3, 6, 12] , got {n_months}')
        
    def resid_mom(self , half_life_ratio = 0.5):
        return (self.model.resid * time_weight(len(self.model.resid) , int(half_life_ratio * len(self.model.resid)))[:,None]).sum()
    
    def r2(self):
        return self.model.r2
    
    def alpha(self):
        return self.model.coef[0]
    
    def resid_vol(self):
        return self.model.resid.std()
    
    def resid_skew(self):
        return self.model.resid.skew()
    
    def resid_kurt(self):
        return self.model.resid.kurt()

@singleton    
class FF3_1m(FamaFrench3):
    N_MONTHS = 1
@singleton
class FF3_2m(FamaFrench3):
    N_MONTHS = 2
@singleton
class FF3_3m(FamaFrench3):
    N_MONTHS = 3    
@singleton
class FF3_6m(FamaFrench3):
    N_MONTHS = 6
@singleton
class FF3_12m(FamaFrench3):
    N_MONTHS = 12

def select_ff3(n_months : int):
    if n_months == 1:
        return FF3_1m
    elif n_months == 2:
        return FF3_2m
    elif n_months == 3:
        return FF3_3m
    elif n_months == 6:
        return FF3_6m
    elif n_months == 12:
        return FF3_12m
    else:
        raise ValueError(f'n_months must be in [1, 2, 3, 6, 12] , got {n_months}')

class mom_ff1m(StockFactorCalculator):
    init_date = 20070101
    category0 = 'behavior'
    category1 = 'momentum'
    description = '1个月ff3残差动量'

    def calc_factor(self , date : int):
        return FamaFrench3.select_ff3(1)(date).resid_mom()
    
class mom_ff2m(StockFactorCalculator):
    init_date = 20070101
    category0 = 'behavior'
    category1 = 'momentum'
    description = '2个月ff3残差动量'

    def calc_factor(self , date : int):
        return FamaFrench3.select_ff3(2)(date).resid_mom()
    
class mom_ff3m(StockFactorCalculator):
    init_date = 20070101
    category0 = 'behavior'
    category1 = 'momentum'
    description = '3个月ff3残差动量'

    def calc_factor(self , date : int):
        return FamaFrench3.select_ff3(3)(date).resid_mom()
    
class mom_ff6m(StockFactorCalculator):
    init_date = 20070101
    category0 = 'behavior'
    category1 = 'momentum'
    description = '6个月ff3残差动量'

    def calc_factor(self , date : int):
        return FamaFrench3.select_ff3(6)(date).resid_mom()
    
class mom_ff12m(StockFactorCalculator):
    init_date = 20070101
    category0 = 'behavior'
    category1 = 'momentum'
    description = '12个月ff3残差动量'

    def calc_factor(self , date : int):
        return FamaFrench3.select_ff3(12)(date).resid_mom()
    
fac = mom_ff12m()
fac.calc_factor(20241107)

secid
1         0.000110
2         0.000611
4         0.001006
5        -0.001931
6         0.001697
            ...   
920019   -0.016095
920066   -0.064374
920088   -0.034169
920099    0.016108
920118    0.001306
Length: 5415, dtype: float64

In [49]:
from src.func.transform import time_weight , apply_ols
ff3.model.resid * time_weight(len(ff3.model.resid) , int(0.5 * len(ff3.model.resid)))[:,None]

secid,1,2,4,6,7,8,9,10,11,12,...,873806,873833,920002,920008,920016,920019,920066,920088,920099,920118
date,,,,,,,,,,,,,,,,,,,,,
20240930,-0.003729,0.003047,-0.000109,0.006718,0.007458,-0.001916,-0.002249,-0.007274,0.001599,0.000188,...,-0.023726,-0.005257,-0.017511,0.074985,0.013393,NaN,NaN,NaN,0.029442,-0.001889
20241008,0.009359,0.007542,-0.003823,-0.043387,0.000573,-0.007732,-0.006428,-0.014934,-0.025940,-0.007052,...,0.013866,-0.003652,0.013313,0.056400,0.029886,NaN,NaN,NaN,0.032298,0.044802
20241009,-0.003853,0.004648,-0.023640,-0.027948,0.019564,-0.000419,0.008344,-0.010061,-0.000526,0.001351,...,-0.010636,-0.000533,-0.019691,0.134339,-0.000928,NaN,NaN,NaN,0.021710,-0.009688
20241010,0.009134,-0.029763,-0.013988,0.015235,0.001571,0.006323,-0.004474,-0.032014,-0.021670,-0.005870,...,0.039016,-0.007011,0.025233,-0.032142,0.057157,NaN,NaN,NaN,0.004296,0.019667
20241011,-0.000982,0.016673,0.088417,0.040450,-0.033595,-0.006187,-0.010448,-0.029658,0.012511,-0.005183,...,0.004453,0.003955,-0.003806,0.044258,0.015568,NaN,NaN,NaN,0.012092,0.006448
20241014,-0.000297,-0.003392,0.049954,-0.039405,0.014472,-0.017785,-0.009593,0.006773,0.014170,-0.003378,...,-0.010486,-0.014179,-0.014132,0.137013,-0.010640,-0.169306,NaN,NaN,-0.018718,-0.013035
20241015,0.014662,0.007117,-0.003038,0.021951,0.012862,-0.008016,-0.014546,0.018872,-0.006673,0.003838,...,-0.001438,-0.025233,-0.023228,-0.057585,-0.039907,-0.091461,NaN,NaN,-0.020580,-0.015316
20241016,-0.001239,0.048383,-0.004413,-0.018605,-0.009184,-0.009778,-0.012264,-0.023547,0.024476,0.002920,...,0.063680,0.035008,0.009801,0.129982,-0.012202,0.026649,NaN,NaN,0.024029,0.022999
20241017,0.009859,-0.045853,0.062378,-0.048883,-0.012429,-0.002390,-0.019468,-0.027381,-0.043548,-0.008463,...,-0.016615,0.007709,0.017544,-0.058275,0.110575,0.216333,NaN,NaN,0.005834,0.039760


In [48]:
time_weight(len(ff3.model.resid) , int(0.5 * len(ff3.model.resid)))[:,None]

array([[0.45853352],
       [0.49144406],
       [0.5267167 ],
       [0.56452098],
       [0.60503861],
       [0.64846432],
       [0.69500685],
       [0.7448899 ],
       [0.79835323],
       [0.8556538 ],
       [0.91706704],
       [0.98288812],
       [1.0534334 ],
       [1.12904196],
       [1.21007721],
       [1.29692865],
       [1.3900137 ],
       [1.4897798 ],
       [1.59670646],
       [1.71130761],
       [1.83413408]])

In [39]:
ff3.model.r2

secid
1         0.904292
2         0.691459
4         0.430139
6         0.283570
7         0.533816
            ...   
920019    0.250012
920066    0.695352
920088    0.351783
920099    0.794215
920118    0.726884
Length: 5363, dtype: float64

In [1]:
import pandas as pd
import numpy as np

from src.factor.classes import StockFactorCalculator
from src.data import TSData
from src.func.transform import apply_ols

date = 20241104
n_months = 1    
min_finite_ratio=0.25
start_date , end_date = TSData.CALENDAR.td_start_end(date , n_months , 'm')
rets = TSData.TRADE.get_returns(start_date , end_date , mask = False , pivot = False)
circ = TSData.TRADE.get_mv(start_date , end_date , mv_type = 'circ_mv' , pivot = False , prev=True)
btop = (1 / TSData.TRADE.get_val_data(start_date , end_date , cols = 'pb' , pivot = False , prev = True)).rename(columns={'pb':'bp'})
rets = rets.merge(circ , on = ['date' , 'secid']).merge(btop , on = ['date' , 'secid'])

rets

,,pctchange,circ_mv,bp
date,secid,,,
20240930,1,0.069177,2.216122e+11,1.858736
20241008,1,0.054873,2.369426e+11,1.738526
20241009,1,-0.093168,2.499444e+11,1.647989
20241010,1,0.048119,2.266576e+11,1.817521
20241011,1,-0.021703,2.324793e+11,1.751313
...,...,...,...,...
20241029,920118,0.059115,3.617049e+08,0.432227
20241030,920118,0.087144,3.830871e+08,0.408097
20241031,920118,-0.096058,4.164711e+08,0.375375


In [2]:
rets['mv_add']= rets['circ_mv'] * rets['pctchange']
rets['mv_rank'] = rets['circ_mv'].groupby('date').rank(pct = True)
rets['bp_rank'] = rets['bp'].groupby('date').rank(pct = True)

rets['mv_group'] = 'M'
rets.loc[rets['mv_rank'] < 0.5 , 'mv_group'] = 'S'
rets.loc[rets['mv_rank'] >= 0.9 , 'mv_group'] = 'B'

rets['bp_group'] = 'M'
rets.loc[rets['bp_rank'] < 1/3 , 'bp_group'] = 'L'
rets.loc[rets['bp_rank'] >= 2/3 , 'bp_group'] = 'H'


In [3]:

def group_ret(df:pd.DataFrame):
    df = df.groupby('date')[['mv_add' , 'circ_mv']].sum()
    return df['mv_add'] / df['circ_mv']

mkt_ret = TSData.TRADE.get_market_return(start_date, end_date)

ret_B = group_ret(rets.loc[rets['mv_group'] == 'B'])
ret_S = group_ret(rets.loc[rets['mv_group'] == 'S'])

ret_H = group_ret(rets.loc[rets['bp_group'] == 'H'])
ret_L = group_ret(rets.loc[rets['bp_group'] == 'L'])

smb = (ret_S - ret_B).rename('smb')
hml = (ret_H - ret_L).rename('hml')

ff3_x = mkt_ret.merge(smb , on = 'date').merge(hml , on = 'date')
ff3_x

,market,smb,hml
date,,,
20240930,0.089301,0.038772,-0.028997
20241008,0.059327,0.043144,-0.037124
20241009,-0.073483,-0.031403,0.003360
20241010,0.007607,-0.008591,0.030683
20241011,-0.029603,-0.015976,0.020430
20241014,0.023957,0.013844,0.000651
20241015,-0.023746,0.015081,-0.001619
20241016,-0.000888,0.009216,0.021814
20241017,-0.008865,0.008642,-0.010372


In [4]:
stk_rets = rets.pivot_table('pctchange' , 'date' , 'secid')
stk_rets


secid,1,2,4,6,7,8,9,10,11,12,...,873806,873833,920002,920008,920016,920019,920066,920088,920099,920118
date,,,,,,,,,,,,,,,,,,,,,
20240930,0.069177,0.099548,0.099865,0.089669,0.100813,0.085586,0.099542,0.101010,0.074398,0.074349,...,0.237636,0.183673,0.179422,0.300000,0.299862,NaN,NaN,NaN,0.299138,0.221607
20241008,0.054873,0.067901,0.100000,-0.017889,0.072378,0.045643,0.077003,0.036697,-0.013238,0.029412,...,0.299220,0.164779,0.229367,0.299825,0.299824,NaN,NaN,NaN,0.299934,0.299773
20241009,-0.093168,-0.100193,-0.099275,-0.100182,-0.013774,-0.095238,-0.070531,-0.101770,-0.100103,-0.077311,...,-0.169542,-0.168877,-0.151650,0.299933,-0.189145,NaN,NaN,NaN,-0.063298,-0.143405
20241010,0.048119,-0.023555,-0.065635,0.054656,-0.011173,0.030702,-0.019751,0.000000,-0.003440,0.005464,...,0.002710,-0.012766,-0.003759,-0.153906,0.028447,NaN,NaN,NaN,-0.093188,-0.027699
20241011,-0.021703,-0.002193,0.100066,0.067179,-0.087571,-0.038298,-0.060445,-0.054187,0.001151,-0.034420,...,-0.099099,-0.060345,-0.086415,0.032712,-0.101204,NaN,NaN,NaN,-0.088942,-0.083368
20241014,0.025597,0.026374,0.100000,-0.016187,0.041796,0.004425,0.013544,0.062500,0.052874,0.016886,...,0.056000,0.045872,0.032425,0.241859,0.036206,-0.300000,NaN,NaN,0.011214,0.028336
20241015,-0.009983,-0.014989,0.012048,0.067642,0.008915,-0.022026,-0.023385,0.009804,-0.019651,-0.016605,...,0.003788,-0.016447,-0.030806,-0.021931,-0.086303,-0.079365,NaN,NaN,-0.032942,-0.030667
20241016,0.013445,0.086957,-0.017857,0.023973,-0.023564,0.009009,-0.013683,0.004854,0.069042,0.013133,...,0.054717,0.085842,-0.009701,0.124787,-0.063480,-0.005603,NaN,NaN,-0.034739,-0.007336
20241017,-0.009121,-0.079000,0.100275,-0.040134,-0.010558,-0.013393,-0.021965,-0.038647,-0.072917,-0.025926,...,0.023256,0.021561,0.053564,0.016251,0.149449,0.299957,NaN,NaN,0.048218,0.076212


In [30]:
alphas

secid,1,2,4,6,7,8,9,10,11,12,...,873806,873833,920002,920008,920016,920019,920066,920088,920099,920118
date,,,,,,,,,,,,,,,,,,,,,
20240930,-0.008133,0.006646,-0.000239,0.014652,0.016264,-0.004179,-0.004906,-0.015863,0.003486,0.000411,...,-0.051744,-0.011464,-0.038189,0.163532,0.029208,NaN,NaN,NaN,0.064209,-0.004120
20241008,0.019043,0.015347,-0.007779,-0.088285,0.001167,-0.015733,-0.013080,-0.030388,-0.052784,-0.014350,...,0.028214,-0.007430,0.027089,0.114764,0.060813,NaN,NaN,NaN,0.065720,0.091163
20241009,-0.007315,0.008824,-0.044882,-0.053062,0.037144,-0.000796,0.015841,-0.019102,-0.000999,0.002565,...,-0.020192,-0.001012,-0.037385,0.255050,-0.001761,NaN,NaN,NaN,0.041218,-0.018393
20241010,0.016181,-0.052723,-0.024779,0.026987,0.002782,0.011200,-0.007926,-0.056709,-0.038387,-0.010399,...,0.069114,-0.012420,0.044697,-0.056936,0.101249,NaN,NaN,NaN,0.007611,0.034839
20241011,-0.001623,0.027557,0.146134,0.066856,-0.055526,-0.010226,-0.017269,-0.049019,0.020678,-0.008567,...,0.007360,0.006537,-0.006291,0.073149,0.025730,NaN,NaN,NaN,0.019986,0.010657
20241014,-0.000458,-0.005231,0.077035,-0.060767,0.022318,-0.027427,-0.014793,0.010444,0.021851,-0.005210,...,-0.016170,-0.021866,-0.021793,0.211289,-0.016409,-0.261087,NaN,NaN,-0.028866,-0.020102
20241015,0.021096,0.010240,-0.004371,0.031585,0.018506,-0.011533,-0.020929,0.027154,-0.009601,0.005523,...,-0.002069,-0.036306,-0.033422,-0.082855,-0.057419,-0.131598,NaN,NaN,-0.029611,-0.022038
20241016,-0.001664,0.064954,-0.005925,-0.024976,-0.012329,-0.013127,-0.016464,-0.031611,0.032859,0.003920,...,0.085489,0.046997,0.013157,0.174499,-0.016381,0.035776,NaN,NaN,0.032259,0.030875
20241017,0.012349,-0.057435,0.078133,-0.061230,-0.015568,-0.002993,-0.024385,-0.034296,-0.054547,-0.010601,...,-0.020811,0.009656,0.021976,-0.072994,0.138504,0.270974,NaN,NaN,0.007307,0.049803


In [33]:
import numpy as np
x , y = ff3_x , stk_rets
np.all(np.isnan(y) , axis = 0) * np.all(np.isnan(x) , axis = 0)

1         NaN
2         NaN
4         NaN
6         NaN
7         NaN
         ... 
920099    NaN
920118    NaN
hml       NaN
market    NaN
smb       NaN
Length: 5366, dtype: object

In [20]:
mkt_ret

,market
date,
20240930,0.089301
20241008,0.059327
20241009,-0.073483
20241010,0.007607
20241011,-0.029603
20241014,0.023957
20241015,-0.023746
20241016,-0.000888
20241017,-0.008865


In [44]:
rets['mv_change'] = rets['pctchange'] * rets['circ_mv']

stk_ret = TSData.TRADE.get_returns(start_date, end_date)
mkt_ret = TSData.TRADE.get_market_return(start_date, end_date)

b = apply_ols(mkt_ret.values.flatten() , stk_ret.values)[1]
b[np.isfinite(stk_ret).sum() < len(mkt_ret) * min_finite_ratio] = np.nan

secid
1        -0.101912
2        -0.594557
4        -0.510529
6        -0.296813
7        -0.085969
            ...   
920019    0.000000
920066    0.000000
920088    0.000000
920099    0.000000
920118    0.000000
Length: 5409, dtype: float64

In [7]:
import numpy as np
import pandas as pd

from src.factor.classes import StockFactorCalculator
from src.data import TSData

def exret_std(date , n_months : int , lag_months : int = 0):
    start_date , end_date = TSData.CALENDAR.td_start_end(date , n_months , 'm' , lag_months)
    exrets = TSData.MODEL.get_exret(start_date , end_date , pivot=True)
    return exrets.std() * np.sqrt(252)

class exret_std1m(StockFactorCalculator):
    init_date = 20070101
    category0 = 'behavior'
    category1 = 'volatility'
    description = '1个月收益率标准差'
    
    def calc_factor(self, date: int):
        return exret_std(date , 1)

from cProfile import Profile
with Profile() as pr:
    v = exret_std1m().calc_factor(20241104) 
# pr.print_stats(1)
v

secid
1         0.188706
2         0.263073
4         0.773525
6         1.062064
7         0.644802
            ...   
873339    0.886305
873527    0.790199
873576    0.946141
873593    1.349053
873665    1.196075
Length: 5150, dtype: float64

In [1]:
from src.api.factor import factor_hierarchy
hier = factor_hierarchy()
hier.factor_df()




,level,file_name,factor_name,init_date,category0,category1,description
0,level0,volatility_skew,price_weiskew6m,20070101,behavior,correlation,12个月贝塔
1,level0,volatility_skew,price_weiskew6m,20070101,behavior,correlation,1个月贝塔
2,level0,volatility_skew,price_weiskew6m,20070101,behavior,correlation,2个月贝塔
3,level0,volatility_skew,price_weiskew6m,20070101,behavior,correlation,3个月贝塔
4,level0,volatility_skew,price_weiskew6m,20070101,behavior,correlation,6个月贝塔
...,...,...,...,...,...,...,...
59,level0,volatility_skew,price_weiskew6m,20070101,behavior,volatility,12个月成交量加权偏度
60,level0,volatility_skew,price_weiskew6m,20070101,behavior,volatility,1个月成交量加权偏度
61,level0,volatility_skew,price_weiskew6m,20070101,behavior,volatility,2个月成交量加权偏度
62,level0,volatility_skew,price_weiskew6m,20070101,behavior,volatility,3个月成交量加权偏度


In [ ]:
date = 20241031
factor_values = hier.test_calc_all_factors(date)

In [8]:
cls = hier.get_factor('turnvp_corr1m')
cls().calc_factor(20241029)

secid
1         0.793899
2         1.321322
4         1.306744
5         0.365365
6         1.030893
            ...   
873339    1.385271
873527    1.393108
873576    1.681234
873593    2.240738
873665    1.610474
Length: 5289, dtype: float64